<a href="https://colab.research.google.com/github/mahenoor22/skin-disease-build/blob/main/detection_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import random
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,Input,BatchNormalization,Dropout
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from glob import glob
from sklearn.model_selection import train_test_split

KeyboardInterrupt: 

# **CallBack**

In [5]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('acc')>0.70):
            print("\nReached 70.0% accuracy so cancelling training!")
            self.model.stop_training = True

## **Import Dataset**

In [6]:
!git clone https://github.com/mahenoor22/skin-disease-build.git

fatal: destination path 'skin-disease-build' already exists and is not an empty directory.


In [7]:
train_dir = r"/content/skin-disease-build/dataset/train"
test_dir=r"/content/skin-disease-build/dataset/test"
os.listdir(train_dir)

['Atopic Dermatitis Photos',
 'Melanoma Skin Cancer Nevi and Moles',
 'Psoriasis pictures Lichen Planus and related diseases',
 'Tinea Ringworm Candidiasis and other Fungal Infections',
 'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions',
 'Nail Fungus and other Nail Disease',
 'Light Diseases and Disorders of Pigmentation',
 'Warts Molluscum and other Viral Infections',
 'Vascular Tumors',
 'Lupus and other Connective Tissue diseases',
 'Eczema Photos',
 'Urticaria Hives',
 'Seborrheic Keratoses and other Benign Tumors',
 'Exanthems and Drug Eruptions',
 'Cellulitis Impetigo and other Bacterial Infections',
 'Bullous Disease Photos',
 'Poison Ivy Photos and other Contact Dermatitis',
 'Acne and Rosacea Photos',
 'Systemic Disease',
 'Hair Loss Photos Alopecia and other Hair Diseases',
 'Scabies Lyme Disease and other Infestations and Bites',
 'Vasculitis Photos',
 'Herpes HPV and other STDs Photos']

# **Image Generator**

In [8]:
data_generator = ImageDataGenerator(validation_split = 0.2)

In [9]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir,
                                                   batch_size=64,
                                                   class_mode='sparse',
                                                   target_size=(150,150))     

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory( test_dir,
                                                   batch_size=64,
                                                   class_mode='sparse',
                                                   target_size=(150,150))


Found 15557 images belonging to 23 classes.
Found 4002 images belonging to 23 classes.


#**Model**

In [10]:
def CNN_model():  
  i = Input(shape = (150,150,3))
  x = Conv2D(32,(3,3),activation='relu')(i)
  #x = MaxPooling2D((2,2))(x)
  #x = BatchNormalization()(x)
  x = Conv2D(32,(3,3),activation='relu')(x)
  #x = BatchNormalization()(x)
  x = MaxPooling2D((2,2))(x)
  x = Dropout(0.25)(x)
  x = Conv2D(64,(3,3),activation='relu')(x)
  #x = BatchNormalization()(x)
  x = Conv2D(64,(3,3),activation='relu')(x)
  #x = BatchNormalization()(x)
  x = MaxPooling2D((2,2))(x)
  x = Dropout(0.25)(x)
  # x = Conv2D(128,(3,3),activation='relu')(x)
  # x = BatchNormalization()(x)
  # x = Conv2D(128,(3,3),activation='relu')(x)
  # x = BatchNormalization()(x)
  # x = MaxPooling2D((2,2))(x)
  x = Flatten()(x)
  x = Dense(512,activation='relu')(x)
  x = Dropout(0.5)(x)
  x = Dense(23,activation='softmax')(x)

  model = Model(i,x)
  model.summary()
  model.compile(optimizer = RMSprop(lr=0.00001),
              loss = 'sparse_categorical_crossentropy',
              metrics = ['acc'])
  history = model.fit_generator(train_generator,validation_data=validation_generator,epochs=30,callbacks=[myCallback()])
  y_preds = model.predict(test_dir)
  print("Classification Report: \n", classification_report(test_dir, y_preds))
    

# **Compile and Run Model**

In [ ]:
CNN_model()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 146, 146, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 73, 73, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 69, 69, 64)        36928 

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
244/244 [==============================] - 1802s 7s/step - loss: 3.0127 - acc: 0.0933 - val_loss: 3.0027 - val_acc: 0.1289
Epoch 2/30
244/244 [==============================] - 1799s 7s/step - loss: 2.9554 - acc: 0.1196 - val_loss: 2.9924 - val_acc: 0.1314
Epoch 3/30
244/244 [==============================] - 1791s 7s/step - loss: 2.9407 - acc: 0.1231 - val_loss: 2.9795 - val_acc: 0.1342
Epoch 4/30
244/244 [==============================] - 1798s 7s/step - loss: 2.9370 - acc: 0.1209 - val_loss: 2.9748 - val_acc: 0.1312
Epoch 5/30
244/244 [==============================] - 1828s 7s/step - loss: 2.9167 - acc: 0.1311 - val_loss: 2.9783 - val_acc: 0.1329
Epoch 6/30
244/244 [==============================] - 1820s 7s/step - loss: 2.9120 - acc: 0.1329 - val_loss: 2.9686 - val_acc: 0.1394
Epoch 7/30
244/244 [==============================] - 1805s 7s/step - loss: 2.9039 - acc: 0.1303 - val_loss: 2.9552 - val_acc: 0.1404
Epoch 8/30
244/244 [==============================] - 1799s 7s

## **GRAPH**

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

KeyError: ignored